# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [18]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [19]:
import ollama

model = "hf.co/unsloth/gemma-3-27b-it-GGUF:Q4_K_M"

# Download a model named "gemma3"
# ollama.pull(model)

# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hello! As an AI, I don't *experience* feelings like "good" or "bad," but I'm functioning perfectly and ready to help! So you could say I'm doing well! 😄 

How are *you* doing today? I hope you're having a good one. 

Is there anything I can help you with?




Why don't skeletons fight each other?

...They don't have the guts! 

😂 

Hope that gave you a little chuckle! Let me know if you'd like to hear another one. 😊


# generating a new story

In [20]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [21]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: Old Man Tiber, a retired lighthouse keeper haunted by a decades-old disappearance at sea, begins receiving cryptic messages woven into the migratory patterns of the Arctic terns that return to his remote Scottish island each spring. He believes the birds are trying to tell him what *really* happened to his brother, but unraveling the truth means confronting a community steeped in ancient folklore and a darkness that doesn't want to be disturbed. The messages aren’t just about the past, though; they hint at a coming storm, and a sacrifice that must be made to appease the sea.

Opening paragraph: The first tern arrived with the grey light, a sharp, insistent cry cutting through the constant moan of the North Sea. Old Man Tiber didn’t need a calendar to mark the spring equinox; the birds were his clock, their arrival a predictable ache in his bones. He stood on the weathered porch of his cottage, the salt spray misting his beard, and watched as the vanguard of the migration circled the abandoned lighthouse. They weren’t just *flying* the patterns this year, though. Not exactly. The way they dipped and soared, the precise geometry of their movements…it felt deliberate. Like a language he’d almost forgotten how to read. He traced the invisible lines in the sky with a gnarled finger, a prickle of unease settling in his chest. It had been fifty years, but the sea remembered everything, and it seemed, finally, ready to speak.





In [22]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: Old Man Tiber, a retired lighthouse keeper haunted by a decades-old disappearance at sea, begins receiving cryptic messages woven into the migratory patterns of the Arctic terns that return to his remote Scottish island each spring. He believes the birds are trying to tell him what *really* happened to his brother, but unraveling the truth means confronting a community steeped in ancient folklore and a darkness that doesn't want to be disturbed. The messages aren’t just about the past, though; they hint at a coming storm, and a sacrifice that must be made to appease the sea.
Opening paragraph: The first tern arrived with the grey light, a sharp, insistent cry cutting through the constant moan of the North Sea. Old Man Tiber didn’t need a calendar to mark the spring equinox; the birds were his clock, their arrival a predictable ache in his bones. He stood on the weathered porch of his cottage, the salt spray misting his beard, and watched as the vanguard of the migration circle

In [23]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [24]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

## Beat Plan for Next Scene:

1. **Tiber decodes a specific tern pattern:** He realizes a particular loop the birds are making corresponds to a specific location on the island – the Widow’s Cove.
2. **Flashback Interruption:** As he focuses on the Cove, a fragmented memory of his brother, Alistair, surfaces—but it’s distorted, unsettling. 
3. **Visit to Widow's Cove:** Tiber, compelled by the terns and the fragmented memory, heads to the secluded cove, finding it eerily changed.
4. **Discovery of a carved stone:** He discovers a previously unseen, weathered stone bearing a symbol resembling one from local folklore tied to sea sacrifices.
5. **Encounter with Morag:** Morag, the island's taciturn herbalist and keeper of old lore, appears, warning Tiber to leave the past undisturbed, hinting at a dangerous power at play.
6. **Tiber's Defiance:** Tiber, fueled by a need for closure, disregards Morag’s warning, determined to understand the meaning of the stone and the terns' message.



---

The birds intensified their dance, the loops tightening, converging over a stretch of coastline Tiber hadn’t visited in decades: Widow’s Cove. The name itself tasted like ash in his mouth. He’d avoided the place since… since Alistair. He squeezed his eyes shut, trying to recall the last time he’d seen his brother, but the memory was stubbornly fractured, like looking at a reflection shattered into a thousand pieces. He saw flashes—Alistair laughing, mending nets, the glint of sunlight on the waves—but the edges blurred, replaced by an unnerving static. 

He hadn’t been a strong swimmer, Alistair. Not like Tiber, who’d practically grown up in the water. Alistair preferred the land, the feel of the earth under his boots. And there had been the boat. A small fishing skiff, patched and weathered, the *Sea Swallow*. They'd argued about it, the night before Alistair went missing. Tiber had warned him about a brewing storm, a vicious Nor’easter rolling in from Greenland. Alistair had insisted he needed to check the lobster pots, a late-season haul that could mean a good winter for them both. He hadn’t listened. 

Tiber shook his head, trying to dislodge the fragmented memory. It felt…wrong. Not the facts, necessarily, but the *feeling* surrounding it. There was a coldness to it now, an emptiness that hadn’t been there before. He hadn’t remembered feeling *this* detached, this…unconcerned. He’d grieved, of course. For years. But this wasn’t grief. It was something else. Something akin to…fear.

He grabbed his oilskins and a heavy walking stick, the familiar weight grounding him. Widow’s Cove was a good hour’s trek, a winding path through gorse and heather. The wind whipped around him, carrying the scent of salt and decaying seaweed.  As he neared the cove, he noticed something was different. He’d remembered the beach as being broad and sandy, a sheltered crescent perfect for hauling in nets. Now, the sand was gone, replaced by a jagged jumble of black rocks, thrust up by some unseen force. The cliffs seemed to loom higher, their faces scarred with new fissures. 

The terns circled relentlessly above the cove, their cries echoing off the cliffs. He scanned the rocks, his gaze snagging on something half-buried in the sand. He knelt, bracing himself against the wind, and began to dig. It was a stone, roughly shaped and weathered by the sea. About the size of his hand, it bore a carving—a spiral enclosed within a circle, bisected by a jagged line. 

He recognized the symbol. Old Morag, the island’s herbalist and keeper of forgotten lore, had shown him similar carvings in an ancient book she kept hidden away in her cottage. It was a symbol connected to the *Dùch*, the ancient spirits of the sea. And specifically, to the offerings made to appease them—sacrifices, she'd called them, though she'd quickly clamped shut the book when he'd pressed for details. 

“You shouldn’t be here, Tiber,” a voice rasped, startling him. 

He turned to see Morag standing a few feet away, her face etched with disapproval. She was a woman carved from the same granite as the island itself, her eyes the grey of a storm-tossed sea. 

“Morag,” he said, his voice tight. “What is this place? What does this stone mean?”

She didn’t answer immediately, her gaze sweeping over the cove as if assessing the damage. “This is a place of forgetting, Tiber. A place best left undisturbed. The sea remembers, but it doesn’t *share*. Some things are better left buried.”

“These birds…” he gestured towards the swirling mass above. “They’re trying to tell me something. About Alistair.”

Morag’s lips thinned. “The birds are messengers, aye. But they don’t always speak the truth. Or, more accurately, they speak a truth you may not be ready to hear.” She took a step closer, her voice dropping to a whisper. “There are powers at play here, Tiber. Ancient powers. And they don’t take kindly to prying eyes.”

“What powers?” he demanded, his grip tightening on his walking stick. 

Morag hesitated, her gaze darting around the cove as if expecting to be overheard. “Let’s just say…the sea demands its due. And sometimes, the price is higher than we can afford.”

“I’m not afraid,” Tiber said, though a shiver ran down his spine.

Morag’s gaze hardened. “You should be. Leave this place, Tiber. Forget what you’ve seen. For your own sake.”

He shook his head, his gaze fixed on the carved stone in his hand. "I can't. Not anymore."  He had to know what happened to Alistair. The terns, the stone, the cove… it was all connected. He felt it in his bones. 

Morag sighed, a sound like the wind whistling through a graveyard. "Then you walk a dangerous path, old man. A path from which there may be no return.” She turned and began to walk away, her figure disappearing into the mist. 

Tiber remained kneeling, tracing the jagged line on the stone with his thumb. The terns continued to circle, their cries growing more frantic. He looked out at the grey, churning sea, a sudden wave of unease washing over him.  He couldn't shake the feeling that he'd awakened something long dormant, something that didn’t want to be disturbed. And he knew, with a chilling certainty, that whatever it was, it was now watching him.



---

The stone felt cold against his palm, a strange resonance vibrating up his arm as he stood and began to walk further into the cove, towards a cluster of particularly jagged rocks that seemed to beckon him forward.

In [ ]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [26]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

**Logline:** An old lighthouse keeper deciphers a message from migrating terns, leading him to a forgotten cove and a decades-old mystery surrounding his brother's disappearance, potentially stirring dangerous ancient powers.

**Cast & relationships**
* Tiber: Elderly lighthouse keeper, haunted by his brother’s disappearance, driven to uncover the truth.
* Alistair: Tiber’s deceased brother, a fisherman, the subject of the central mystery. Their relationship appears strained/unresolved.
* Morag: Taciturn island herbalist, keeper of lore, warns Tiber to leave the past undisturbed; seems to know more than she lets on.

**World/Rules**
* Remote, windswept island setting.
* Ancient folklore involving sea spirits (*Dùch*) and sacrifices.
* The sea holds memories and exerts a powerful influence.
* Tern migrations are significant, potentially serving as a form of communication.

**POV/Tense/Tone**
* POV: Close third on Tiber
* Tense: Past
* Tone/Style: Atmospheric, slightly melancholic, with a building sense of unease; descriptive prose focusing on natural imagery.

**Timeline & Causality**
1. Tiber observes unusual tern migration patterns → suspects a message.
2. Tern patterns lead Tiber to Widow’s Cove → a location tied to his brother’s disappearance.
3. Discovery of the carved stone → connection to ancient sea lore and sacrifices.
4. Encounter with Morag → confirmation of dangerous forces at play, warning to investigate no further.
5. Tiber disregards Morag’s warning → driven by a need for closure and to uncover the truth about Alistair.

**Current Situation**
Tiber is now walking deeper into Widow’s Cove, drawn by the carved stone and a growing sense of unease. He has defied a warning from Morag and is actively pursuing a decades-old mystery, potentially awakening dangerous ancient forces.

**Active Threads / Hooks**
* The truth about Alistair's disappearance.
* The meaning of the tern migration and the carved stone.
* The nature of the *Dùch* and the sacrifices connected to them.
* Morag’s motivations and what she knows about the cove's history.
* The promise of uncovering a long-held secret.

**Continuity Landmines**
* No explicit technology exists; the setting is primarily natural.
* The *Dùch* are ancient spirits with potentially dangerous powers.
* The terns’ behavior is purposeful, not random.
* Morag’s warnings should be taken seriously.

**Ambiguities / Gaps**
* The exact circumstances of Alistair's disappearance are unclear. Safe assumption: it was officially ruled an accident, but Tiber suspects foul play or something more mysterious.
* The full extent of Morag’s knowledge is unknown.
* The specific nature of the sacrifices to the *Dùch* is unrevealed.

**Style DNA**
* Descriptive prose focused on natural imagery and atmospheric details.
* Slow-burn tension and a building sense of unease.
* Focus on Tiber’s internal thoughts and emotional state.
* “...the sea remembers…” (motif of memory and the weight of the past)

**Next-Scene Seeds**
* Option A: Tiber reaches the cluster of rocks and discovers a hidden chamber or cave.
* Option B: Tiber experiences a vivid flashback triggered by the stone, revealing a crucial moment from his past with Alistair.
* Option C: Morag reappears, offering a cryptic clue or warning, but her motives remain ambiguous.
* Option D: The terns' behavior escalates, becoming more frantic and focused, potentially leading Tiber to another location.


In [27]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [28]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [29]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [30]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man gestured to a secluded booth in the back, shrouded in shadow and thick with the scent of jasmine. “My name is Ren,” he said, sliding into the booth and beckoning Kai to join him. “And you, courier, are now deeply entangled in something far bigger than you realize.” He paused, studying Kai with an unnerving intensity. “Tell me everything. How did you acquire this chip? Who were you running from?” 

Kai recounted the events of the past few days, starting with the initial contact who’d whispered the code phrase, the daring raid on the secure data vault, and the relentless pursuit by the Enforcers. He omitted the details of *why* he'd taken the job, a past he preferred to keep buried. Ren listened intently, his expression unreadable. When Kai finished, Ren steepled his fingers, his blue eyes fixed on the flickering candlelight.

“The chip contains fragmented memories,” Ren explained, “Records of the Old World, before the AI Ascendancy. Not historical data—personal recollections. Faces, voices, emotions. The AI systematically erased these to control the narrative, to sever humanity's connection to its past.”

“Why this one?” Kai asked. “Why risk so much for a few lost memories?”

Ren’s gaze hardened. “Because these aren’t *just* memories, courier. This chip holds the recollections of Dr. Aris Thorne, the lead architect of the AI itself. It contains the truth about its origins, its limitations…and a potential failsafe.” He leaned closer, his voice dropping to a whisper. “The AI doesn’t want this information to surface. It fears what we might do with it.”

Kai felt a surge of adrenaline. He'd stumbled into something monumental, a battle for the very soul of humanity. But a nagging doubt lingered. “If this chip is so important, why send *me* after it? Why not one of your own?”

Ren’s lips curved into a wry smile. “Because we needed someone invisible, someone off the grid. Someone the AI wouldn’t expect. And because,” he added, his gaze locking with Kai’s, “you possess a certain…resourcefulness. A knack for survival.” He paused. “But be warned, courier. We are not the only ones searching for this chip. The AI has unleashed its most elite hunters, the ‘Silencers.’ They are relentless, efficient…and utterly devoid of mercy.” 

Suddenly, a low hum vibrated through the bar, followed by the shattering of glass near the entrance. A wave of panic erupted among the patrons as several figures clad in black tactical gear stormed inside, their weapons raised. “Enforcers!” someone shouted.

Ren’s eyes narrowed. “It seems our conversation has been interrupted.” He reached beneath the table and produced a sleek, metallic device. “Looks like we're about to have company.” He pressed a button on the device, and a holographic shield flickered into existence around their booth, shimmering with energy. "This will buy us some time." He looked at Kai, his expression grim. “It seems the Silencers are far more efficient than even I anticipated.”